In [ ]:
# Cell 1 - Load OpenAI API key from .env
from dotenv import load_dotenv
import os

import openai
from openai import OpenAI
import re

import json
import streamlit as st

✅ API key loaded successfully.


In [ ]:


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY:
    print("✅ API key loaded successfully.")
else:
    print("❌ Failed to load API key. Please check your .env file.")


In [3]:
#GPT_MODEL="gpt-4"
#GPT_MODEL="gpt-3.5-turbo"
GPT_MODEL = "gpt-4o-mini"



In [4]:
print("OpenAI library version:", openai.__version__)

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    organization=os.getenv("OPENAI_ORG_ID")
)

OpenAI library version: 1.60.1


In [5]:
def call_openai_with_prompt(prompt: str, client, model="gpt-3.5-turbo", temperature=0.7):
    """
    Sends the prompt to the OpenAI Chat API and returns the simplified response.
    """
    print("\n**** Sending message to OpenAI ****\n\n", prompt, "\n")
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )

    return response.choices[0].message.content



In [6]:
prompt = "Why do I get headache when I don't sleep? I often work overnight and my sleep schedule is messed up as I get only 3 hours of sleep some days? Is that a health risk? If yes how serious?"
print(prompt)

Why do I get headache when I don't sleep? I often work overnight and my sleep schedule is messed up as I get only 3 hours of sleep some days? Is that a health risk? If yes how serious?


In [7]:
call_openai_with_prompt(prompt, client)


**** Sending message to OpenAI ****

 Why do I get headache when I don't sleep? I often work overnight and my sleep schedule is messed up as I get only 3 hours of sleep some days? Is that a health risk? If yes how serious? 



"Headaches due to lack of sleep, especially when you consistently get only 3 hours of sleep, can occur for several reasons:\n\n1. **Sleep Deprivation**: Not getting enough sleep can lead to increased muscle tension and stress, which can trigger headaches.\n\n2. **Circadian Rhythm Disruption**: Working overnight and having an irregular sleep schedule can disrupt your body's natural circadian rhythms, leading to various symptoms, including headaches.\n\n3. **Increased Stress**: Lack of sleep can increase stress levels and anxiety, both of which can contribute to headache development.\n\n4. **Dehydration and Hunger**: Sometimes, when you don’t sleep enough, you might forget to stay hydrated or eat properly, which can also lead to headaches.\n\n5. **Migraines**: For some individuals, lack of sleep can act as a trigger for migraine headaches.\n\nAs for health risks, chronic sleep deprivation can lead to several serious health issues, including:\n\n- **Cognitive Impairment**: Difficulty conc

In [ ]:
# Cell 3 - PoC 2: Transcribe audio using Whisper CLI (run this in terminal/shell)
# Save this cell content for reference, but execute in terminal
"""
# Command to run in terminal after installing whisper and ffmpeg:
# Replace 'symptom_audio.wav' with your actual audio file

whisper symptom_audio.wav --model base
"""


In [ ]:
# Cell 4 - PoC 3: Multi-step clarifying question + triage reasoning
clarification_chain = client.chat.completions.create(
    model=GPT_MODEL,
    messages=[
        {"role": "system", "content": "You are a medical assistant."},
        {"role": "user", "content": "I have a headache and slight nausea."},
        {"role": "assistant", "content": "Do you also have blurred vision or sensitivity to light?"},
        {"role": "user", "content": "Yes, both."},
        {"role": "assistant", "content": "Thank you. Based on your symptoms and responses, you may have a migraine. "
                                         "You should monitor at home if symptoms are manageable, but see a doctor if they worsen or persist."}
    ]
)




Clarification + Triage Response:



In [9]:
print("Clarification + Triage Response:\n")
print(clarification_chain.choices[0].message.content)

Clarification + Triage Response:

Thank you for sharing your symptoms. Given that you have a headache, slight nausea, blurred vision, and sensitivity to light, it’s possible that you may be experiencing a migraine. Here are a few suggestions that might help:

1. **Rest in a Dark Room**: Find a quiet, dark place to rest, as light and noise can worsen migraine symptoms.

2. **Stay Hydrated**: Drink water to avoid dehydration, which can exacerbate headaches.

3. **Cold Compress**: Apply a cold compress or ice pack to your forehead for relief.

4. **Medication**: Over-the-counter pain relief, such as ibuprofen or acetaminophen, may help. Follow dosing instructions on the label.

5. **Relaxation Techniques**: Try deep breathing, meditation, or gentle stretching exercises to relieve tension.

However, if your symptoms worsen or if you experience severe pain, confusion, difficulty speaking, or any other concerning symptoms, you should seek medical attention right away. It’s always best to con

In [ ]:


# Load API key and organization ID
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_ID")

# Create a client
client = openai.Client()

# Define the symptoms that need guidelines
symptoms = [
    "Fever"
]

# Function to generate guideline using GPT-3.5 Turbo (client-based)
def generate_guideline(symptom):
    prompt = f"""
    For the symptom "{symptom}", provide:
    1. Clarifying questions (comma-separated)
    2. Red flag conditions (comma-separated)
    3. Triage options (Emergency, Doctor visit, Monitor at home)
    4. Safe advice for each triage level (what to do or when to seek care)

    Please return a structured response as plain text, separated by '---' for each section. For example:

    Clarifying Questions: [Question 1, Question 2]
    Conditions to Flag: [Condition 1, Condition 2]
    Triage Recommendations: [Emergency: Text, Doctor Visit: Text, Monitor at Home: Text]
    Safe Advice: [Text]

    Avoid any extra formatting or code blocks.
    """
    
    # Chat-based API call using client
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Correct way to access the content from the response object
    raw_response = response.choices[0].message.content
    
    print(f"Raw response for {symptom}: {raw_response}")
    
    return raw_response

# Generate and save guidelines for "Fever" only
def generate_and_save_guidelines(symptoms):
    symptom_guidelines = []
    
    for symptom in symptoms:
        print(f"Generating guidelines for: {symptom}")
        raw_response = generate_guideline(symptom)

        # Extract information using regex or string operations
        clarifying_questions = re.findall(r'Clarifying Questions:\s*(.*?)\s*---', raw_response, re.DOTALL)
        conditions_to_flag = re.findall(r'Conditions to Flag:\s*(.*?)\s*---', raw_response, re.DOTALL)
        triage_recommendation = re.findall(r'Triage Recommendations:\s*(.*?)\s*---', raw_response, re.DOTALL)
        safe_advice = re.findall(r'Safe Advice:\s*(.*?)$', raw_response, re.DOTALL)

        # Clean and split the extracted data into individual items (if needed)
        clarifying_questions = clarifying_questions[0].split('\n') if clarifying_questions else []
        conditions_to_flag = conditions_to_flag[0].split('\n') if conditions_to_flag else []
        triage_recommendation = triage_recommendation[0].split('\n') if triage_recommendation else []
        safe_advice = safe_advice[0].split('\n') if safe_advice else []

        # You can now structure this data as needed
        guideline = {
            "symptom": symptom,
            "clarifying_questions": clarifying_questions,
            "conditions_to_flag": conditions_to_flag,
            "triage_recommendation": triage_recommendation,
            "safe_advice": safe_advice
        }

        symptom_guidelines.append(guideline)

    # Save to a text file or CSV
    with open("symptom_guidelines_fever.txt", "w") as f:
        f.write(str(symptom_guidelines))

# Run the function for "Fever" only
generate_and_save_guidelines(symptoms)


Generating guidelines for: Fever
Raw response for Fever: Clarifying Questions: [What is your current temperature?, How long have you had the fever?, Are you experiencing any other symptoms such as chills, headache, or rash?, Have you traveled recently or been exposed to anyone who is sick?, Are you taking any medications or have any underlying health conditions?]

Conditions to Flag: [Sepsis, Meningitis, Pneumonia, Influenza, COVID-19, Serious infection]

Triage Recommendations: [Emergency: Call 911 or go to the nearest emergency room if experiencing difficulty breathing, persistent chest pain, confusion, or seizures, Doctor Visit: Schedule an appointment with your doctor if the fever persists for more than 3 days or is higher than 103°F (39.4°C) with no clear cause, Monitor at Home: Keep track of your symptoms and fever; seek care if symptoms worsen or persist for more than a few days]

Safe Advice: [For emergency situations, call for immediate help; for doctor visits, try to avoid co

In [ ]:
# Load the symptom guidelines JSON file
with open('../data/combined_symptom_database.json', 'r') as file:
    symptom_guidelines = json.load(file)
# Print the loaded guidelines
print("Loaded symptom guidelines:")
for guideline in symptom_guidelines:
    print(guideline)


Loaded symptom guidelines:
tension_headache
migraine
cluster_headache
thunderclap_headache
rebound_headache
cough
nasal_congestion
runny_nose
sneezing
postnasal_drip
fever
chills
fatigue
facial_pain_or_pressure
ear_fullness_or_pain
blurred_vision
dizziness
nausea_vomiting_with_headache
neck_stiffness
confusion_or_altered_mental_status
headache_after_bending_over
headache_after_physical_exertion
headache_after_sexual_activity
valsalva_maneuver_induced_headache


In [13]:
def get_symptom_guidelines(symptom):
    """
    Given a symptom, fetch its corresponding data (questions, conditions, etc.) from the symptom database.
    """
    symptom = symptom.lower()
    if symptom in symptom_guidelines:
        return symptom_guidelines[symptom]
    else:
        return None  # If symptom not found
    print(f"Guidelines for {symptom}:")
    print(f"Clarifying Questions: {guideline['clarifying_questions']}")
    print(f"Conditions to Flag: {guideline['conditions_to_flag']}")
    print(f"Triage Recommendations: {guideline['triage_recommendation']}")
    print(f"Safe Advice: {guideline['safe_advice']}")
# Example usage


In [14]:
symptom = "headache"  # Example input from the user
symptom_data = get_symptom_guidelines(symptom)

if symptom_data:
    prompt = f"""
    For the symptom "{symptom}", generate the following:
    1. Clarifying questions to ask the patient.
    2. Red flag conditions that need to be flagged.
    3. Triage options (Emergency, Doctor visit, Monitor at home).
    4. Safe advice for each triage level (what to do or when to seek care).

    Clarifying Questions: {symptom_data['clarifying_questions']}
    Conditions to Flag: {symptom_data['conditions_to_flag']}
    Triage Recommendations: {symptom_data['triage_recommendations']}
    Safe Advice: {symptom_data['safe_advice']}
    """

print("Prompt for OpenAI:")
print(prompt)

Prompt for OpenAI:
Why do I get headache when I don't sleep? I often work overnight and my sleep schedule is messed up as I get only 3 hours of sleep some days? Is that a health risk? If yes how serious?


In [15]:
def generate_guideline_prompt(symptom):
    """
    Given a symptom, generate a structured GPT-4 prompt using the symptom's details.
    """
    symptom_data = get_symptom_guidelines(symptom)

    if symptom_data is None:
        return f"Sorry, we don't have guidelines for {symptom}."

    # Construct the prompt using the structured data
    prompt = f"""
    For the symptom "{symptom}", generate the following:
    1. Clarifying questions to ask the patient.
    2. Red flag conditions that need to be flagged.
    3. Triage options (Emergency, Doctor visit, Monitor at home).
    4. Safe advice for each triage level (what to do or when to seek care).

    Clarifying Questions: {symptom_data['clarifying_questions']}
    Conditions to Flag: {symptom_data['conditions_to_flag']}
    Triage Recommendations: {symptom_data['triage_recommendations']}
    Safe Advice: {symptom_data['safe_advice']}
    """
    return prompt


In [ ]:
# # Load API key and organization ID from environment variables
# load_dotenv()
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.organization = os.getenv("OPENAI_ORG_ID")

# def get_gpt_response(prompt):
#     """
#     Get GPT response for the given prompt.
#     """
#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",  # Using GPT-3.5 turbo
#         messages=[
#             {"role": "system", "content": "You are a helpful medical assistant."},
#             {"role": "user", "content": prompt}
#         ]
#     )
#     return response.choices[0].message.content


In [ ]:
def display_result(symptom):
    prompt = generate_guideline_prompt(symptom)
    print(f"Generated Prompt: {prompt}")  # Debugging line
    if "Sorry" in prompt:
        st.write(prompt)
    else:
        gpt_response = get_gpt_response(prompt)
        print(f"GPT Response: {gpt_response}")  # Debugging line
        if gpt_response:
            st.write(gpt_response)
        else:
            st.write("No response generated.")


In [ ]:
# # Function to display the result for the entered symptom
# def display_result(symptom):
#     prompt = generate_guideline_prompt(symptom)  # Generate prompt for symptom
#     if "Sorry" in prompt:  # Check if symptom was found
#         st.write(prompt)
#     else:
#         gpt_response = get_gpt_response(prompt)  # Get response from GPT-4
#         st.write(gpt_response)

# # Streamlit UI setup
# st.title("Medical Symptom Advisor")
# symptom_input = st.text_input("Enter a symptom:", "")


In [ ]:
# Title of the app
st.title('Streamlit Test App')

# Display a simple text
st.write("Hello, Streamlit is working!")

# You can add more interactive widgets, such as sliders or buttons
number = st.slider("Pick a number", 0, 100)
st.write(f"Selected number: {number}")
